<a href="https://colab.research.google.com/github/vinayakgautam368/FACE-UNLOCK/blob/main/FACE_UNLOCK248.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import glob
import numpy as np
import random
import matplotlib.pyplot as plt
from PIL import Image
from itertools import combinations
from collections import OrderedDict
from tqdm import tqdm
  
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader
import torchvision
import torchvision.transforms as transforms


In [2]:
torch.cuda.is_available()

False

In [3]:
PATH="/content/drive/MyDrive/archive/"

In [4]:
class CustomDataset(Dataset):

    def __init__(self, path, transform=transforms.ToTensor()):
        self.dirs = glob.glob(f'{path}*/')
        self.transform = transform
        
        self.total_images = self._get_images(self.dirs, self.transform)

    def _get_images(self, dirs, transform):
        ti = []
        for dir in tqdm(dirs):
            images = [transform(Image.open(image)) for image in glob.glob(f'{dir}*')]
            ti.append(images)
        return ti
        
    def __len__(self):
        return self.dirs.__len__()
    
    def __getitem__(self,idx):
        return torch.stack(self.total_images[idx], dim=0)

In [5]:
process = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=0.4422, std=0.1931),
])


In [6]:
att_dataset = CustomDataset(PATH, transform=process)

100%|██████████| 40/40 [00:01<00:00, 23.73it/s]


In [7]:
# att_dataset[0].shape


In [8]:
# _,axxr = plt.subplots(40,10, figsize=(12,60))
# for i in range(40):
#     for j in range(10):
#         axxr[i][j].imshow(att_dataset[i][j].squeeze(), cmap='gray')
# plt.setp(plt.gcf().get_axes(), xticks=[], yticks=[])

In [9]:

dataset = list(att_dataset)
# for i in dataset:
#   print(i.shape)

In [10]:

def triplet_loss(anchor, positive, negative, margin=1):
    pos_dist = (anchor - positive).pow(2).sum(-1) #.pow(.5)
    neg_dist = (anchor - negative).pow(2).sum(-1) #.pow(.5)
    loss = F.relu(pos_dist - neg_dist + margin)
    return loss.mean()

For the triplets we select one class as anchor class fromehich we select one random image from same class as positive,and for the negative we select 1 random image from each (n-1)classes ,here we have 40 classes so foe each anchor we have 39 negatives(1 from each 39 classes),and each class contain 10 images so for each class we have (39)mul(10)=390 tripets ,so for 40 classes we have (390)mul(40)=15600 triplets.

In [11]:
def get_random_triplets(embeddings,  targets=None) -> torch.Tensor:


    triplets = []
    for i, embedding in enumerate(embeddings):
        temp = embeddings.pop(i)

        for anchor in embedding:
            positive = random.choice(embedding)

            for negatives in embeddings:
                negative = random.choice(negatives)

                triplets.append(torch.stack([anchor, positive, negative], dim=0))

        embeddings.insert(i, temp)

    return torch.stack(triplets, dim=0)

In [12]:
def dist(enc1,enc2):
    return (enc1 - enc2).pow(2).sum(-1).pow(.5)

In [13]:

# triplets = get_random_triplets(dataset[:5])
# print(triplets.shape)

In [14]:
# _,axxr = plt.subplots(60,3, figsize=(12,120))
# for i in range(60):
#     for j in range(3):
#         axxr[i][j].imshow(triplets[i][j].squeeze(), cmap='gray')
# plt.setp(plt.gcf().get_axes(), xticks=[], yticks=[])
# # ref : https://stackoverflow.com/questions/25124143/matplotlib-subplots-get-rid-of-tick-labels-altogether
# plt.show()

In [15]:
train = dataset[0:30]
len(train)

30

In [16]:
# test = dataset[30:]
# len(test)

In [17]:
class BasicBlock(nn.Module):     # THIS IS THE FIRST CONVOLUTION BLOCK
  def __init__(self,inplanes,planes,stride=1,downsample=None):
    super().__init__()
    self.conv1=nn.Conv2d(inplanes,planes,kernel_size=3,padding=1,stride=stride,bias=False)
    self.bn1=nn.BatchNorm2d(planes)
    self.relu=nn.ReLU(inplace=True)
 
    self.conv2=nn.Conv2d(planes,planes,kernel_size=3,padding=1,stride=1,bias=False)
    self.bn2=nn.BatchNorm2d(planes)
    self.downsample=downsample
    self.stride=stride
 
  def forward(self,x):
 
    identity=x
 
    out=self.conv1(x)
    out=self.bn1(out)
    out=self.relu(out)
 
    out=self.conv2(out)
    out=self.bn2(out)
 
    if self.downsample is not None:
      identity=self.downsample(x)
 
    out+=identity
    out=self.relu(out)
      
 
    return out

In [18]:
class ResNet(nn.Module):
  def __init__(self,block,numbers,classes=128):
    super().__init__()
 
    #this is first simple conv block
    self.inplanes=64
    self.conv1=nn.Conv2d(1,self.inplanes,kernel_size=7,stride=2,padding=3,bias=False)    
    self.bn1=nn.BatchNorm2d(self.inplanes)
    self.relu=nn.ReLU(inplace=True)
    self.maxpool=nn.MaxPool2d(kernel_size=3,stride=2,padding=1)       #gives shape  (2,64,56,56)
 
    # now 4 layers
    self.layer1=self._make_layer(block,64,numbers[0])
    self.layer2=self._make_layer(block,128,numbers[1],stride=2)
    self.layer3=self._make_layer(block,256,numbers[2],stride=2)
    self.layer4=self._make_layer(block,512,numbers[3],stride=2)
 
    self.avgpool=nn.AdaptiveAvgPool2d((1,1))
    self.fc=nn.Linear(512,classes)
 
  def _make_layer(self,block,planes,blocks,stride=1):
    downsample=None
 
    if stride!=1 or self.inplanes!=planes:
      downsample=nn.Sequential(nn.Conv2d(self.inplanes,planes,1,stride,bias=False),nn.BatchNorm2d(planes))
    layers=[]
 
    layers.append(block(self.inplanes,planes,stride,downsample))
    self.inplanes=planes
 
    for j in range(1,blocks):
      layers.append(block(self.inplanes,planes))
    return nn.Sequential(*layers)
 
 
  def semi_forward(self,x):
    x=self.conv1(x)
    x=self.bn1(x)
    x=self.relu(x)
    x=self.maxpool(x)
    # print(x.shape)
 
    x=self.layer1(x)
    x=self.layer2(x)
    x=self.layer3(x)
    x=self.layer4(x)
 
    x=self.avgpool(x)
    x=torch.flatten(x,1)
 
    x=self.fc(x)
 
 
    return x


  def forward(self,triplet):
    anc = self.semi_forward(triplet[:,0,...])
    pos = self.semi_forward(triplet[:,1,...])
    neg = self.semi_forward(triplet[:,2,...])
    return [anc, pos, neg]    

In [19]:
numbers=[2,2,2,2]

resnet18 = ResNet(BasicBlock,numbers)

# t = torch.rand(10,1,224,224)
# print('rand', t.shape)
# t = resnet18.semi_forward(t)
# print('Inbuilt', t.shape)

In [20]:
# t = torch.rand(100,3,1,224,224)
# print('rand', t.shape)
# t = resnet18(t)
# print('resnet18', t[0].shape, t[1].shape, t[2].shape)

In [21]:

def Train(model, batch, loss_fn, optimizer, cost):
    model.train()

    apn = model(batch)

    optimizer.zero_grad()
    loss = loss_fn(*apn)
    cost.append(loss.item())
    loss.backward()
    optimizer.step()

    return cost[-1]

In [22]:

def Evaluate(model, batch):
    model.eval()
    def dist(enc1,enc2):
        return (enc1 - enc2).pow(2).sum(-1).pow(.5)
    with torch.no_grad():
        sample = torch.cat([model.semi_forward(imgs[0].unsqueeze(0)) for imgs in batch])
        total_enc = [model.semi_forward(img) for img in batch]
        pred = [torch.stack([dist(enc,sample).argmin() for enc in total_enc[i]]) for i in range(len(total_enc))]
        acc = sum([(pred[i] == i).sum() for i in range(len(total_enc))])
        del total_enc
    return (acc.item() / (len(batch) * 10) )

In [23]:

torch.set_grad_enabled(True)
resnet18.train(True)

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [24]:
learning_rate = 0.0001
optimizer = optim.Adam(resnet18.parameters(), lr = learning_rate)
torch_triplet_loss = nn.TripletMarginLoss()
if torch.cuda.is_available():
    resnet18 = resnet18.cuda()

In [ ]:
cost = [float('inf')]
train_acc = [0]
test_acc = [0]
epochs = 5
for epoch in range(epochs):

    triplets = get_random_triplets(train)
    loader = DataLoader(triplets, batch_size=100)
    for i,batch in enumerate(loader):

        loss = Train(resnet18, batch, triplet_loss, optimizer, cost)
        
        # pred = Evaluate(resnet18, train)
        # acc1 = ( (pred == torch.arange(len(pred)).reshape(-1,1)).sum() / (len(pred)*10) ).item()
        # train_acc.append(acc1)

        # pred = Evaluate(resnet18, test)
        # acc2 = ( (pred == torch.arange(len(pred)).reshape(-1,1)).sum() / (len(pred)*10) ).item()
        # test_acc.append(acc2)

        if (i+1)%20==0 :
            # print(f'Epoch:[{epoch+1}/{epochs}], Step:[{i+1}/87]', 'Cost : {:.2f}, Train Acc: {:.2f}, Test Acc: {:.2f}'.format(loss, acc1, acc2))
            print(f'Epoch:[{epoch+1}/{epochs}], Step:[{i+1}/87]', 'Cost : {:.2f}'.format(loss))

Epoch:[1/5], Step:[20/87] Cost : 0.06
Epoch:[1/5], Step:[40/87] Cost : 0.04
Epoch:[1/5], Step:[60/87] Cost : 0.07
Epoch:[1/5], Step:[80/87] Cost : 0.01


In [ ]:
# cost = []
# train_acc = []
# test_acc = []
# total_acc = []
# epochs = 10
# for epoch in range(epochs):

#     triplets = get_random_triplets(train)
#     loader = DataLoader(triplets, batch_size=100)
#     for i,batch in enumerate(loader):

#         apn = resnet18(batch)

#         optimizer.zero_grad()
#         loss = triplet_loss(*apn)
#         cost.append(loss.item())

#         loss.backward()
#         optimizer.step()

#         ####
#         with torch.no_grad():
#             resnet18.train(False)
#             sample = torch.cat([resnet18.semi_forward(att_dataset[i][0].unsqueeze(0)) for i in range(40)])
#             total_enc = [resnet18.semi_forward(img) for img in dataset]
#             pred = [torch.stack([dist(enc,sample).argmin() for enc in total_enc[i]]) for i in range(40)]
#             acc1 = sum([(pred[i] == i).sum() for i in range(0,30)])
#             acc2 = sum([(pred[i] == i).sum() for i in range(30,40)])
#             train_acc.append(acc1.item()  / 300)
#             test_acc.append(acc2.item() / 100)
#             total_acc.append((acc1.item() + acc2.item()) / 400 )
#             del total_enc
#             resnet18.train(True)
#         ####

#         if (i+1)%20==0 :
#             print(f'Epoch:[{epoch+1}/{epochs}], Step:[{i+1}/87]', 'Cost : {:.2f}, Train Acc: {:.2f}, Test Acc: {:.2f}'.format(loss.item(), acc1.item()/3, acc2.item()))

In [ ]:
a=[[1,2,3],[12,3],[3,4,4],4]
for i,j in enumerate(a):
  a.pop(i)
  print(i,j)
  print(a)
  break